# Something

In [ ]:
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep


In [ ]:
###To start, set the path to open your image. In this example the image bands are in a folder called "Image" in the 
###working directory. B* means that the script will read in all band tifs collectively rather than having to code each one.
image_path = glob("Image\LC08_L1TP_207023_20211122_20211130_01_T1_B*.tif")
###Ensure the bands are sorted in order
image_path.sort()
###Stack the bands into a single image. Give the no data values. WHY? WHAT IS META? ARR_ST?
image, meta = es.stack(image_path, nodata=-9999)



In [ ]:
###plot your image. Default plot below is the false colour composite (4, 3, 2,) but these can be changed. Str_clip will 
###impact the amount of image stretch applied.
ep.plot_rgb(image, rgb = [4, 3, 2], title = "test", stretch=True, str_clip=0.1)

In [ ]:
###Function to calculate the ndvi based on the image that you open and stack, above.
def ndvi (image):
    ###Here we define the two bands used to calculate the NDVI: red band and near infrared (NIR) band
    ###These are changed to floats AND WHY IS THAT?
    red = image[3].astype(float)
    nir = image[4].astype(float)
    
    ###and calculate the ndvi using the formula and numpy's divide function
    my_ndvi = np.divide((nir-red), (nir+red))
    
    #give your plot a name
    plot_title = ["My plot name"]
    
    ###Depending on your area of interest you might like to change vmin and vmax. 
    ###If, for example, your values range from -0.3 to 0.4, try using e.g. -0.4 and 0.5 as vmin/vmax to get a clearer 
    ###idea of the actual spread of NDVI values. As NDVI values can mathematically range from -1 to 1 these are used as 
    ###default vmin/vmax.
    ###Feel free to change the colour map, too. RdYlGn (continuous scale of red -> green) is a commonly used one 
    ###with NDVI's, where darker greens represent healthier vegetation. 
    ###Choose more here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    ep.plot_bands(my_ndvi, cmap = "RdYlGn", title = plot_title, vmin = -1, vmax = 1)
    
    ###and create the classes you want on your ndvi. Edit these as you please. -np.inf and np.inf will take the negative
    ###and positive extremities. This will create 5 classes, but you can add more (just remember to assign them a colour below)
    ndvi_classes = [-np.inf, 0, 0.15, 0.3, 0.5, np.inf]
    ###Below we take the output array for our ndvi calculated above and give each value the integer value of the 
    ###class it falls into
    image_ndvi_class = np.digitize(my_ndvi, ndvi_classes)
    ### Apply a 'no data' mask to the array. This will remove no value data entries
    image_ndvi_class = np.ma.masked_where(np.ma.getmask(my_ndvi), image_ndvi_class)
    np.unique(image_ndvi_class)
    
    ###define colour map and class names
    ###turn colours list into a colourmap (cmp) to read into imshow, below.
    ndvi_colours = ["blue", "green", "yellow", "red", "pink"]
    ndvi_cmap = ListedColormap(ndvi_colours)
    ndvi_class_names = [
        "No Vegetation",
        "Bare Earth",
        "Low Density Vegetation",
        "Moderate Density Vegetation",
        "High Density Vegetation"
    ]
    
    # Get list of classes from the masked array
    classes = np.unique(image_ndvi_class)
    classes = classes.tolist()
    ### The mask returns a value of 'NONE' as a class. This will give an error message that there are more classes (n+1) than
    ###colours in cmap (n). We can remove this. (The example in the function has n = 5 classes).
    classes = classes[0:5]

    # Plot your data. Customise aspects such as figure size and font size.
    fig, ax = plt.subplots(figsize=(14, 14))
    im = ax.imshow(image_ndvi_class, cmap=ndvi_cmap)

    ep.draw_legend(im_ax=im, classes=classes, titles=ndvi_class_names)
    ax.set_title(
        "My Plot Title",
        fontsize=16,
    )
    ax.set_axis_off()

    # Auto adjust subplot to fit figure size
    plt.tight_layout()

In [ ]:
###run the function on your image. This will give you the plotted output for the NDVI.
ndvi(image)